In [1]:

import os
import keras

# import pillow
from PIL import Image

# for prediction
import numpy as np
import pandas as pd

from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
# from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, model_from_yaml
from keras.optimizers import Adam



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:

base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.



/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [3]:

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation


In [4]:

model=Model(inputs=base_model.input,outputs=preds)


In [5]:

# Print a summary representation of your model
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [7]:

for layer in model.layers[:87]:
    layer.trainable=False


for layer in model.layers[87:]:
    layer.trainable=True


train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies


model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [8]:

# paperspace path
training_data_path = '/storage/train/'


train_generator=train_datagen.flow_from_directory(training_data_path,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)



Found 9568 images belonging to 3 classes.


In [9]:

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer; loss function will be categorical cross entropy; evaluation metric will be accuracy

step_size_train = train_generator.n//train_generator.batch_size



In [10]:

# Start Training !
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   # use_multiprocessing=True,
                   # workers = 8,
                   # epochs = 10)
                   epochs = 30)
                   # epochs = 100)



W0927 18:59:30.039504 140561151125312 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/30
149/149 [==============================] - 167s 1s/step - loss: 0.5096 - accuracy: 0.7918
Epoch 2/30
149/149 [==============================] - 151s 1s/step - loss: 0.3765 - accuracy: 0.8334
Epoch 3/30
149/149 [==============================] - 150s 1s/step - loss: 0.3610 - accuracy: 0.8393
Epoch 4/30
149/149 [==============================] - 151s 1s/step - loss: 0.3338 - accuracy: 0.8486
Epoch 5/30
149/149 [==============================] - 149s 1000ms/step - loss: 0.3163 - accuracy: 0.8589
Epoch 6/30
149/149 [==============================] - 148s 996ms/step - loss: 0.3007 - accuracy: 0.8678
Epoch 7/30
149/149 [==============================] - 149s 1s/step - loss: 0.2826 - accuracy: 0.8747
Epoch 8/30
149/149 [==============================] - 149s 1s/step - loss: 0.2524 - accuracy: 0.8888
Epoch 9/30
149/149 [==============================] - 149s 1s/step - loss: 0.2370 - accuracy: 0.8941
Epoch 10/30
149/149 [==============================] - 150s 1s/step - loss: 0.2242 -

In [11]:
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)


In [12]:

test_generator = test_datagen.flow_from_directory(
    # directory='./test/',
    directory='/storage/test/',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False
)


Found 30 images belonging to 1 classes.


In [13]:

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=STEP_SIZE_TEST,
                            verbose=1)


predicted_class_indices=np.argmax(pred,axis=1)


30/30 [==============================] - 1s 33ms/step


In [14]:

# train_datagen wird benötigt
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

# oder als Dict definieren (gleicher Task)
labels = {0: 'Detailbilder', 1: 'Hauptbilder', 2: 'Zimmerbilder'}

predictions = [labels[k] for k in predicted_class_indices]



In [15]:
predictions

['Hauptbilder',
 'Detailbilder',
 'Detailbilder',
 'Hauptbilder',
 'Detailbilder',
 'Detailbilder',
 'Zimmerbilder',
 'Hauptbilder',
 'Zimmerbilder',
 'Detailbilder',
 'Detailbilder',
 'Detailbilder',
 'Detailbilder',
 'Zimmerbilder',
 'Zimmerbilder',
 'Hauptbilder',
 'Detailbilder',
 'Zimmerbilder',
 'Detailbilder',
 'Detailbilder',
 'Hauptbilder',
 'Zimmerbilder',
 'Detailbilder',
 'Detailbilder',
 'Detailbilder',
 'Detailbilder',
 'Zimmerbilder',
 'Zimmerbilder',
 'Hauptbilder',
 'Detailbilder']

In [16]:

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})


In [17]:
results.to_csv("results.csv",index=False)

In [17]:

## Save
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)



In [18]:

# serialize model to YAML
model_yaml = model.to_yaml()
with open('model.yaml', 'w') as yaml_file:
    yaml_file.write(model_yaml)



In [18]:

# serialize weights to HDF5
model.save_weights("model.h5")
# print("Saved model to disk")

